![title](../images/header.png)

Add attributes from polygons and assets
-------
This notebook lets you add additional columns to your cleaned CEO data, including administrative areas for sub-national estimates and updated strata from assets in GEE. 
###### For more information contact aurelie.shapiro@fao.org or remi.dannunzio@fao.org

In [114]:
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt

import geopandas
import rasterio
import matplotlib.pyplot as plt
from shapely.geometry import Point

import ee
import geemap
import pandas as pd

In [115]:
# Initialize Earth Engine
ee.Initialize()

##### read points from scripts 1-7

In [116]:
#points = pd.read_csv('/home/sepal-user/module_results/esbae/DRC/DRC_all_points_whisp.csv', delimiter=',', low_memory=False)
#points = pd.read_csv('/home/sepal-user/module_results/esbae/DRC/DRC_all_ceo_2015_2022_2023_allData_whisp_prov_biome.csv', delimiter=',', low_memory=False)
#points = pd.read_csv('/home/sepal-user/module_results/esbae/GAB/GAB_all_ceo_2015_2022_2023_clean.csv', delimiter=',', low_memory=False)
points = pd.read_csv('/home/sepal-user/module_results/esbae/COG/COG_all_ceo_2015_2016_2022_2023_clean.csv',delimiter=',', low_memory=False)
#points = pd.read_csv('/home/sepal-user/module_results/esbae/EQG/EQG_all_ceo_2015_2022_2023_prov.csv',delimiter=',', low_memory=False)
#points = pd.read_csv('/home/sepal-user/module_results/esbae/EQG/EQG_all_ceo_2015_2022_2023_prov.csv',delimiter=',', low_memory=False)
#points = pd.read_csv('/home/sepal-user/module_results/esbae/CAR/CAR_all_ceo_2015_2022_2023_pref.csv',delimiter=',', low_memory=False)
#points = pd.read_csv('/home/sepal-user/module_results/esbae/CMR/CMR_all_ceo_2015_2022_2023_clean.csv',delimiter=',', low_memory=False)
#read GEE points
#points = ee.FeatureCollection('users/faocongo/sbae/sbae_hex16_cmr')

In [117]:
len(points)

9359

In [118]:
all_columns = points.columns.tolist()

# Display the list of column names
print(all_columns)

['UID', 'UID.1', 'ID', 'plotid', 'point_id', 'sampleid', 'lon', 'lat', 'sample_geom', 'Ref_FNF_2015', 'Ref_Regeneration', 'Ref_Change_1522', 'Ref_Change_Type_1522', 'ArtFor', 'ArtMine', 'InfraR', 'Urb', 'IndFor', 'Other', 'Other_Desc', 'IndMine', 'IndAg', 'ArtAg', 'Ref_Year_1522', 'Ref_NFtype_2015', 'Ref_Ftype_2015', 'collection', 'interpreted', 'Ref_LCover_2015', 'Ref_Change_Year_1522', 'Ref_Change_LCover_1522', 'Def2016', 'Def2017', 'Def2018', 'Def2019', 'Def2020', 'Def2021', 'Def2022', 'Deg2016', 'Deg2017', 'Deg2018', 'Deg2019', 'Deg2020', 'Deg2021', 'Deg2022', 'Defall', 'Degall', 'Stable', 'NF', 'DensFor', 'DensDryFor', 'SecFor', 'DryOpenFor', 'Mangrove', 'Swamp', 'Gallery', 'Plantation', 'Woodland', 'Shrubland', 'Grassland', 'Aquatic', 'Bare', 'Cultivated', 'Builtup', 'Water', 'DensFor_Def', 'DensDryFor_Def', 'SecFor_Def', 'DryOpenFor_Def', 'Mangrove_Def', 'Swamp_Def', 'Gallery_Def', 'Plantation_Def', 'Woodland_Def', 'DensFor_Deg', 'DensDryFor_Deg', 'SecFor_Deg', 'DryOpenFor_Deg',

In [119]:
#drop any columns if needed
#esbae = esbae.drop('index_right', axis=1)

In [120]:
#enter Lat and Lon columns
LATcol = 'lat'
LONcol = 'lon'

In [121]:
gdf = geopandas.GeoDataFrame(points, geometry=geopandas.points_from_xy(points[LONcol], points[LATcol]), crs="EPSG:4326")

#### get strata from model file

In [122]:
model = pd.read_csv('/home/sepal-user/module_results/esbae/COG/COG_esbae_2015_2022_model.csv',delimiter=',', low_memory=False)

In [123]:
model['stratum'].value_counts(dropna=False)

stratum
1    206000
2     64687
3     20908
Name: count, dtype: int64

In [124]:
model_columns = model.columns.tolist()

# Display the list of column names
print(model_columns)

['images', 'mon_images', 'bfast_change_date', 'bfast_magnitude', 'bfast_means', 'cusum_change_date', 'cusum_confidence', 'cusum_magnitude', 'red_mean', 'red_sd', 'red_min', 'red_max', 'nir_mean', 'nir_sd', 'nir_min', 'nir_max', 'swir1_mean', 'swir1_sd', 'swir1_min', 'swir1_max', 'swir2_mean', 'swir2_sd', 'swir2_min', 'swir2_max', 'ndfi_mean', 'ndfi_sd', 'ndfi_min', 'ndfi_max', 'brightness_mean', 'brightness_sd', 'brightness_min', 'brightness_max', 'greenness_mean', 'greenness_sd', 'greenness_min', 'greenness_max', 'wetness_mean', 'wetness_sd', 'wetness_min', 'wetness_max', 'bs_slope_mean', 'bs_slope_sd', 'bs_slope_max', 'bs_slope_min', 'ccdc_change_date', 'ccdc_magnitude', 'aspect', 'dw_class_mode', 'dw_tree_prob__max', 'dw_tree_prob__min', 'dw_tree_prob__stdDev', 'dw_tree_prob_mean', 'elevation', 'esa_lc20', 'esa_lc21', 'esri_lc17', 'esri_lc18', 'esri_lc19', 'esri_lc20', 'esri_lc21', 'gfc_gain', 'gfc_loss', 'gfc_lossyear', 'gfc_tc00', 'lang_tree_height', 'potapov_tree_height', 'slope'

In [125]:
stratum_gdf = gdf.merge(
    model[['point_id', 'stratum']],  # Keep only 'point_id' and 'stratum' columns from model_df
    on='point_id',  # Merge on point_id
    how='left'  # Keep all rows from points_gdf, and only matching rows from model_df
)

In [126]:
stratum_gdf.head()

,UID,UID.1,ID,plotid,point_id,sampleid,lon,lat,sample_geom,Ref_FNF_2015,...,Ref_Driver2_2023,Ref_Fire2_2023,Ref_LCover_2022,Ref_Change_Year_2023,Ref_Change_LCover_2023,Def2023,Deg2023,interpreted_qc,geometry,stratum
0,0,0,0,542426,542426,542426,11,-4,POINT(11.279983 -3.903345),1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (11.27998 -3.90334),1
1,1,1,1,1016750,1016750,1016750,13,-4,POINT(12.605415 -4.298105),1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (12.60542 -4.2981),1
2,2,2,2,1016273,1016273,1016273,13,-4,POINT(12.605014 -4.182788),1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (12.60501 -4.18279),2
3,3,3,3,1015146,1015146,1015146,13,-4,POINT(12.568966 -3.865546),1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (12.56897 -3.86555),1
4,4,4,4,1006758,1006758,1006758,18,3,POINT(17.626291 2.90407),1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (17.62629 2.90407),2


In [127]:
#drop any columns if needed
columns_to_drop = ['UID.1']
stratum_gdf = stratum_gdf.drop(columns_to_drop, axis=1)

In [128]:
strat_columns = stratum_gdf.columns.tolist()
# Display the list of column names
print(strat_columns)

['UID', 'ID', 'plotid', 'point_id', 'sampleid', 'lon', 'lat', 'sample_geom', 'Ref_FNF_2015', 'Ref_Regeneration', 'Ref_Change_1522', 'Ref_Change_Type_1522', 'ArtFor', 'ArtMine', 'InfraR', 'Urb', 'IndFor', 'Other', 'Other_Desc', 'IndMine', 'IndAg', 'ArtAg', 'Ref_Year_1522', 'Ref_NFtype_2015', 'Ref_Ftype_2015', 'collection', 'interpreted', 'Ref_LCover_2015', 'Ref_Change_Year_1522', 'Ref_Change_LCover_1522', 'Def2016', 'Def2017', 'Def2018', 'Def2019', 'Def2020', 'Def2021', 'Def2022', 'Deg2016', 'Deg2017', 'Deg2018', 'Deg2019', 'Deg2020', 'Deg2021', 'Deg2022', 'Defall', 'Degall', 'Stable', 'NF', 'DensFor', 'DensDryFor', 'SecFor', 'DryOpenFor', 'Mangrove', 'Swamp', 'Gallery', 'Plantation', 'Woodland', 'Shrubland', 'Grassland', 'Aquatic', 'Bare', 'Cultivated', 'Builtup', 'Water', 'DensFor_Def', 'DensDryFor_Def', 'SecFor_Def', 'DryOpenFor_Def', 'Mangrove_Def', 'Swamp_Def', 'Gallery_Def', 'Plantation_Def', 'Woodland_Def', 'DensFor_Deg', 'DensDryFor_Deg', 'SecFor_Deg', 'DryOpenFor_Deg', 'Mangrov

In [129]:
# Rename columns
column_names = {
   'stratum': 'eSBAE_strat_1522'
#    'NAME_2': 'Territoire',
#    'Unnamed: 0':'Index'
}

# Use the rename() method to rename columns
stratum_gdf.rename(columns=column_names, inplace=True)

In [130]:
stratum_gdf['eSBAE_strat_1522'].value_counts(dropna=False)

eSBAE_strat_1522
2      3235
NaN    3227
3      2132
1       765
Name: count, dtype: int64

In [131]:
# save output
out_file =  '/home/sepal-user/module_results/esbae/COG/COG_all_ceo_2015_2016_2022_2023_clean_eSBAE_strat.csv'
stratum_gdf.to_csv(out_file,index=False)

#### add attributes from local shapefile to associate points with sub-national juridictions

In [132]:
# this shapefile should be uploaded to your SEPAL workspace
#poly = "/home/sepal-user/data/admin/RDC_Province_26.shp"
#poly = "/home/sepal-user/data/aoi/Ecoregions2017.shp"
poly = "/home/sepal-user/data/admin/COG_adm1.shp"
#poly = "/home/sepal-user/data/admin/cmr_admbnda_adm1_inc_20180104.shp"
poly_shp = gpd.read_file(poly)
poly_shp.head()

,Superficie,Duparteme,ID,geometry
0,1,Pointe noire,1,"POLYGON ((11.82223 -4.78274, 11.82249 -4.77831..."
1,3,Brazzaville,2,"POLYGON ((15.13816 -4.34912, 15.13908 -4.33897..."
2,1,Kouilou,3,"POLYGON ((11.14631 -3.94915, 11.16561 -3.92881..."
3,1,Bouenza,4,"POLYGON ((12.59318 -3.81769, 12.60018 -3.81474..."
4,3,Pool,5,"POLYGON ((13.87014 -3.53576, 13.8884 -3.52374,..."


In [133]:
# from the table above, identify the column name you want to associate
#admin_name = 'ADM1_FR'
#admin_name = 'shapeName'
#admin_name = 'admin1Name'
#admin_name = 'NOM'
admin_name = 'Duparteme'
#new_name = 'Prefecture'
new_name = 'Departement'

In [134]:
poly_crs = poly_shp.crs
print("Current CRS:", poly_crs)

Current CRS: EPSG:4326


In [135]:
#poly_shp = poly_shp.set_crs(epsg=3395)

In [136]:
poly_shp = poly_shp.to_crs(gdf.crs)

In [137]:
# Reproject both GeoDataFrames to the same projected CRS (e.g., EPSG:3395 for World Mercator)
gdf = stratum_gdf
gdf = gdf.to_crs("EPSG:3395")
poly_shp = poly_shp.to_crs("EPSG:3395")

# Step 1: Perform the spatial join for intersecting points
joined_data_intersecting = gpd.sjoin(gdf, poly_shp[[admin_name, 'geometry']], how="left", predicate="within", lsuffix='left', rsuffix='right')

# Drop the 'index_right' column if it exists, to avoid conflicts
if 'index_right' in joined_data_intersecting.columns:
    joined_data_intersecting = joined_data_intersecting.drop(columns='index_right')

# Step 2: Identify points without an intersection (NaN values in the Admin_Name column)
no_intersection_points = joined_data_intersecting[joined_data_intersecting[admin_name].isna()]

# Step 3: Perform the nearest spatial join for points without an intersection
nearest_join = gpd.sjoin_nearest(no_intersection_points.drop(columns=admin_name), poly_shp[[admin_name, 'geometry']], how="left", distance_col="distance_to_polygon")

# Drop the 'index_right' column if it exists in the nearest join result
if 'index_right' in nearest_join.columns:
    nearest_join = nearest_join.drop(columns='index_right')

# Step 4: Combine intersecting and nearest joined data into one GeoDataFrame
# Retain only the specified `admin_name` column from `poly_shp`
joined_data_combined = pd.concat([
    joined_data_intersecting.dropna(subset=[admin_name]),
    nearest_join
])

# Select only the columns from `gdf` plus `admin_name`
columns_to_keep = list(gdf.columns) + [admin_name]
joined_data = joined_data_combined[columns_to_keep]

# Display the resulting GeoDataFrame
joined_data.head()

,UID,ID,plotid,point_id,sampleid,lon,lat,sample_geom,Ref_FNF_2015,Ref_Regeneration,...,Ref_Fire2_2023,Ref_LCover_2022,Ref_Change_Year_2023,Ref_Change_LCover_2023,Def2023,Deg2023,interpreted_qc,geometry,eSBAE_strat_1522,Duparteme
0,0,0,542426,542426,542426,11,-4,POINT(11.279983 -3.903345),1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (1255681.964 -431948.27),1,Kouilou
1,1,1,1016750,1016750,1016750,13,-4,POINT(12.605415 -4.298105),1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (1403228.379 -475712.195),1,Kouilou
2,2,2,1016273,1016273,1016273,13,-4,POINT(12.605014 -4.182788),1,non,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (1403183.74 -462925.629),2,Niari
3,3,3,1015146,1015146,1015146,13,-4,POINT(12.568966 -3.865546),1,oui,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (1399170.895 -427758.92),1,Niari
4,4,4,1006758,1006758,1006758,18,3,POINT(17.626291 2.90407),1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (1962149.739 321254.859),2,Likouala


In [138]:
nearest_join.head()

,UID,ID,plotid,point_id,sampleid,lon,lat,sample_geom,Ref_FNF_2015,Ref_Regeneration,...,Ref_LCover_2022,Ref_Change_Year_2023,Ref_Change_LCover_2023,Def2023,Deg2023,interpreted_qc,geometry,eSBAE_strat_1522,Duparteme,distance_to_polygon
209,209,209,743912,743912,743912,14,0,POINT(13.931462 0.038683),1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,POINT (1550843.256 4277.345),1,Cuvette ouest,47
268,268,268,637848,637848,637848,14,1,POINT(14.107285 0.551082),1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,POINT (1570415.783 60936.445),1,Cuvette ouest,247
279,279,279,445810,445810,445810,13,2,POINT(13.299947 2.094921),1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,POINT (1480543.328 231696.694),1,Sangha,344
300,300,300,616667,616667,616667,12,-3,POINT(11.648782 -2.883875),1,non,...,NaN,NaN,NaN,NaN,NaN,NaN,POINT (1296736.481 -319018.922),1,Niari,4351
519,519,107,1435725,1435725,1435725,15,-4,POINT(15.45215855331443 -4.18934773521767),0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,POINT (1720126.422 -463652.933),2,Pool,6382


In [139]:
joined_data[admin_name].value_counts(dropna=False)

Duparteme
Pool             1485
Cuvette          1366
Plateaux         1316
Niari             941
Likouala          924
Sangha            857
Cuvette ouest     748
Lokoumou          634
Bouenza           592
Kouilou           476
Brazzaville        15
Pointe noire        6
Name: count, dtype: int64

In [140]:
province_pts = pd.pivot_table(joined_data,values='plotid',index=[admin_name],columns=['collection'],aggfunc="count",margins=True,
                             margins_name='Total',dropna=False)
province_pts

collection,QC_2023,coll_1522_1,coll_1622_1,coll_1622_2,coll_1622_3,coll_2023_1,coll_2023_2,Total
Duparteme,,,,,,,,
Bouenza,54,20,45,158,211,12,92,592
Brazzaville,NaN,NaN,1,3,7,1,3,15
Cuvette,127,41,136,271,466,60,265,1366
Cuvette ouest,84,42,80,146,214,23,159,748
Kouilou,77,17,42,76,120,18,126,476
Likouala,88,62,155,142,243,60,174,924
Lokoumou,78,29,60,99,141,27,200,634
Niari,107,36,86,184,288,32,208,941
Plateaux,115,54,128,284,472,33,230,1316


In [141]:
# Remove rows where the 'Province' column has NaN values
joined_data = joined_data.dropna(subset=[admin_name])

In [142]:
len(joined_data)

9360

In [143]:
poly_shp = poly_shp.to_crs("EPSG:3395")

In [144]:
#calculate area of poly
poly_shp['area_sqm'] = poly_shp.geometry.area

In [145]:
poly_shp['area_ha'] = poly_shp['area_sqm'] / 10000

In [146]:
pd.set_option('display.float_format', lambda x: '%.0f' % x)

# Display the GeoDataFrame with the new area columns
print(poly_shp[[admin_name, 'area_ha']])

        Duparteme  area_ha
0    Pointe noire    10706
1     Brazzaville    27842
2         Kouilou  1375365
3         Bouenza  1254749
4            Pool  3445542
5        Lokoumou  2108334
6           Niari  2747918
7        Plateaux  3875611
8         Cuvette  4337083
9   Cuvette ouest  3164015
10         Sangha  5790719
11       Likouala  6213810


In [147]:
# Rename columns
column_mapping = {
   admin_name: new_name
#    'NAME_2': 'Territoire',
#    'Unnamed: 0':'Index'
}

# Use the rename() method to rename columns
joined_data.rename(columns=column_mapping, inplace=True)

In [148]:
list_columns = joined_data.columns.tolist()

# Display the list of column names
print(list_columns)

['UID', 'ID', 'plotid', 'point_id', 'sampleid', 'lon', 'lat', 'sample_geom', 'Ref_FNF_2015', 'Ref_Regeneration', 'Ref_Change_1522', 'Ref_Change_Type_1522', 'ArtFor', 'ArtMine', 'InfraR', 'Urb', 'IndFor', 'Other', 'Other_Desc', 'IndMine', 'IndAg', 'ArtAg', 'Ref_Year_1522', 'Ref_NFtype_2015', 'Ref_Ftype_2015', 'collection', 'interpreted', 'Ref_LCover_2015', 'Ref_Change_Year_1522', 'Ref_Change_LCover_1522', 'Def2016', 'Def2017', 'Def2018', 'Def2019', 'Def2020', 'Def2021', 'Def2022', 'Deg2016', 'Deg2017', 'Deg2018', 'Deg2019', 'Deg2020', 'Deg2021', 'Deg2022', 'Defall', 'Degall', 'Stable', 'NF', 'DensFor', 'DensDryFor', 'SecFor', 'DryOpenFor', 'Mangrove', 'Swamp', 'Gallery', 'Plantation', 'Woodland', 'Shrubland', 'Grassland', 'Aquatic', 'Bare', 'Cultivated', 'Builtup', 'Water', 'DensFor_Def', 'DensDryFor_Def', 'SecFor_Def', 'DryOpenFor_Def', 'Mangrove_Def', 'Swamp_Def', 'Gallery_Def', 'Plantation_Def', 'Woodland_Def', 'DensFor_Deg', 'DensDryFor_Deg', 'SecFor_Deg', 'DryOpenFor_Deg', 'Mangrov

In [149]:
joined_data = joined_data.to_crs("EPSG:4326")

In [150]:
# save output
out_file_shp_att =  '/home/sepal-user/module_results/esbae/CMR/CMR_all_ceo_2015_2016_2022_2023_prov_eSBAE_strat.csv'
joined_data.to_csv(out_file_shp_att,index=False)

#### assign strata from GEE asset

In [151]:
gdf = joined_data

In [155]:
#if you don't have a unique ID field
gdf['UID_1'] = range(1, len(gdf) + 1)

In [156]:
UID = 'UID_1'

In [157]:
# Assuming df is your DataFrame - check for unique values
if gdf[UID].nunique() == len(gdf):
    print("UID has entirely unique values.")
else:
    print("UID does not have entirely unique values.")

if gdf['ID'].nunique() == len(gdf):
    print("ID has entirely unique values.")
else:
    print("ID does not have entirely unique values.")

UID has entirely unique values.
ID does not have entirely unique values.


In [158]:
# Remove duplicates keeping the last occurrence (the most recent date)
#gdf.drop_duplicates(subset='sample_geom', keep='last', inplace=True)

In [159]:
# select 2022-2023 strata asset you want to sample 
#asset2223 = ee.Image('users/faocongo/sbae/EQG_FNF_kmeans_strat_5_2022_2023')
#asset2223 = ee.Image('users/faocongo/sbae/CAF_FNF_kmeans_strat_5_2022_2023')
#asset2223 = ee.Image('users/faocongo/sbae/GAB_FNF_kmeans_strat_5_2022_2023')
#asset2223 = ee.Image('users/faocongo/sbae/CMR_FNF_kmeans_strat_5_2022_2023')
#asset2223 = ee.Image('users/faocongo/sbae/COD_allData_stratification_2023')
#asset2223 = ee.Image('users/faocongo/sbae/COD_FNF_kmeans_strat_5_2022_2023')
#asset2223 = ee.Image('users/faocongo/sbae/COD_allData_stratification_2023')
asset2223 = ee.Image('projects/ee-ameliearquero/assets/COG_FNF_kmeans_strat_3_2022_2023')
#asset2223 = ee.Image('users/faocongo/sbae/CMR_allData_stratification_2023')

In [160]:
# what to rename sampled band to. it is always called first
newName2223 = 'TNT_stratum_2023'

In [161]:
# select 2015-2022 strata asset you want to sample 
#asset1522 = ee.Image('users/faocongo/sbae/EQG_FNF_kmeans_strat_5_2015_2022')
#asset1522 = ee.Image('users/faocongo/sbae/CAF_FNF_kmeans_strat_5_2015_2022')
#asset1522 = ee.Image('users/faocongo/sbae/GAB_FNF_kmeans_strat_5_2015_2022')
#asset1522 = ee.Image('users/faocongo/sbae/COD_allData_stratification_1522')
#asset1522 = ee.Image('users/faocongo/sbae/COD_allData_stratification_1522')
#asset1522 = ee.Image('users/faocongo/sbae/CMR_FNF_kmeans_strat_5_2015_2022')
#asset1522 = ee.Image('users/faocongo/sbae/COD_FNF_kmeans_strat_5_2015_2022')

In [162]:
newName1522 = 'TNT_stratum_1522'

In [165]:
#resolution of asset
scale = 30

In [166]:
#name of band
band = 'strata'

In [167]:
#sample 2223 stratum
gdf = gdf.reset_index(drop=True)
#with just a few columns
#this can take a little while with many points
dfs = []
for i in range(0, len(points), 5000):
    tmp_df = gdf.loc[i:i+4999]
    tmp_fc = geemap.gdf_to_ee(tmp_df[[UID, 'geometry']])
    tmp_smp = asset2223.select(band).reduceRegions(**{
        'collection': tmp_fc,
        'scale': scale,
        'reducer': ee.Reducer.first()   
    })

    tmp_res = geemap.ee_to_gdf(tmp_smp)
    dfs.append(tmp_res)
    
sampled_df = pd.concat(dfs)

In [168]:
sampled_df

,geometry,UID_1,first
0,POINT (11.27998 -3.90334),1,1
1,POINT (12.60542 -4.2981),2,1
2,POINT (12.60501 -4.18279),3,1
3,POINT (12.56897 -3.86555),4,1
4,POINT (17.62629 2.90407),5,1
...,...,...,...
4355,POINT (11.77561 -3.70443),9356,3
4356,POINT (15.43095 -4.11819),9357,2
4357,POINT (15.3995 -4.24333),9358,3
4358,POINT (18.06696 1.54308),9359,2


In [169]:
sampled_df = sampled_df.rename(columns={'first':newName2223})
sampled_df.head()

,geometry,UID_1,TNT_stratum_2023
0,POINT (11.27998 -3.90334),1,1
1,POINT (12.60542 -4.2981),2,1
2,POINT (12.60501 -4.18279),3,1
3,POINT (12.56897 -3.86555),4,1
4,POINT (17.62629 2.90407),5,1


###### only if you have 2 strata layers to join

In [97]:
#sample 1522 stratum
gdf = sampled_df.reset_index(drop=True)
#with just a few columns
#this can take a little while with many points
dfs = []
for i in range(0, len(points), 5000):
    tmp_df = gdf.loc[i:i+4999]
    tmp_fc = geemap.gdf_to_ee(tmp_df[[UID, 'geometry',newName2223]])
    tmp_smp = asset1522.select(band).reduceRegions(**{
        'collection': tmp_fc,
        'scale': scale,
        'reducer': ee.Reducer.first()   
    })

    tmp_res = geemap.ee_to_gdf(tmp_smp)
    dfs.append(tmp_res)
    
resampled_df = pd.concat(dfs)

In [98]:
resampled_df = resampled_df.rename(columns={'first':newName1522})
resampled_df.head()

,geometry,TNT_stratum_2023,UID,TNT_stratum_1522
0,POINT (11.96675 7.39501),3,0,3
1,POINT (13.46219 4.61658),1,1,1
2,POINT (13.32633 4.47217),1,2,1
3,POINT (13.4541 4.94322),3,3,1
4,POINT (13.02145 4.51968),1,4,2


##### recover columns

In [170]:
# Step 1: Merge the two DataFrames on the 'ID' column (assuming 'ID' is the common key)
#merged_df = pd.merge(resampled_df, points, on=UID, how='left', suffixes=('', '_dup'))
#merged_df = pd.merge(sampled_df, joined_data, on=UID, how='left', suffixes=('', '_dup'))
merged_df = pd.merge(sampled_df, gdf, on=UID, how='left', suffixes=('', '_dup'))
# Step 2: Remove duplicate columns
# Here, we assume that any duplicate column from df2 will have the '_dup' suffix added by the merge
# We will keep the columns from df1 and drop the duplicate ones from df2.
merged_df = merged_df.loc[:, ~merged_df.columns.str.endswith('_dup')]
merged_df.head()

,geometry,UID_1,TNT_stratum_2023,UID,ID,plotid,point_id,sampleid,lon,lat,...,Ref_Driver2_2023,Ref_Fire2_2023,Ref_LCover_2022,Ref_Change_Year_2023,Ref_Change_LCover_2023,Def2023,Deg2023,interpreted_qc,eSBAE_strat_1522,Departement
0,POINT (11.27998 -3.90334),1,1,0,0,542426,542426,542426,11,-4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Kouilou
1,POINT (12.60542 -4.2981),2,1,1,1,1016750,1016750,1016750,13,-4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Kouilou
2,POINT (12.60501 -4.18279),3,1,2,2,1016273,1016273,1016273,13,-4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,Niari
3,POINT (12.56897 -3.86555),4,1,3,3,1015146,1015146,1015146,13,-4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Niari
4,POINT (17.62629 2.90407),5,1,4,4,1006758,1006758,1006758,18,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,Likouala


In [171]:
merged_df_cols = merged_df.columns.tolist()
# Display the list of column names
print(merged_df_cols)

['geometry', 'UID_1', 'TNT_stratum_2023', 'UID', 'ID', 'plotid', 'point_id', 'sampleid', 'lon', 'lat', 'sample_geom', 'Ref_FNF_2015', 'Ref_Regeneration', 'Ref_Change_1522', 'Ref_Change_Type_1522', 'ArtFor', 'ArtMine', 'InfraR', 'Urb', 'IndFor', 'Other', 'Other_Desc', 'IndMine', 'IndAg', 'ArtAg', 'Ref_Year_1522', 'Ref_NFtype_2015', 'Ref_Ftype_2015', 'collection', 'interpreted', 'Ref_LCover_2015', 'Ref_Change_Year_1522', 'Ref_Change_LCover_1522', 'Def2016', 'Def2017', 'Def2018', 'Def2019', 'Def2020', 'Def2021', 'Def2022', 'Deg2016', 'Deg2017', 'Deg2018', 'Deg2019', 'Deg2020', 'Deg2021', 'Deg2022', 'Defall', 'Degall', 'Stable', 'NF', 'DensFor', 'DensDryFor', 'SecFor', 'DryOpenFor', 'Mangrove', 'Swamp', 'Gallery', 'Plantation', 'Woodland', 'Shrubland', 'Grassland', 'Aquatic', 'Bare', 'Cultivated', 'Builtup', 'Water', 'DensFor_Def', 'DensDryFor_Def', 'SecFor_Def', 'DryOpenFor_Def', 'Mangrove_Def', 'Swamp_Def', 'Gallery_Def', 'Plantation_Def', 'Woodland_Def', 'DensFor_Deg', 'DensDryFor_Deg',

In [172]:
#drop any columns if needed
#columns_to_drop = ['sample_geom']
#merged_df = merged_df.drop(columns_to_drop, axis=1)

In [173]:
#merged_df[newName1522].value_counts(dropna=False)

In [174]:
merged_df[newName2223].value_counts(dropna=False)

TNT_stratum_2023
1      4724
2      3026
3      1465
NaN     145
Name: count, dtype: int64

In [175]:
# Using loc to replace only rows where 'stratum' is 0
#merged_df.loc[merged_df[newName1522] == 0, newName1522] = 1

In [176]:
# Using loc to replace only rows where 'stratum' is 0
merged_df.loc[merged_df[newName2223] == 0, newName2223] = 1

In [177]:
#merged_df[newName1522] = merged_df[newName1522].fillna(1)
merged_df[newName2223] = merged_df[newName2223].fillna(1)

In [178]:
print(merged_df.dtypes)

geometry            geometry
UID_1                  int64
TNT_stratum_2023     float64
UID                    int64
ID                     int64
                      ...   
Def2023              float64
Deg2023              float64
interpreted_qc       float64
eSBAE_strat_1522     float64
Departement           object
Length: 140, dtype: object


In [90]:
pd.pivot_table(
    merged_df,
    values=UID,
    index=[newName1522],
    columns=['collection'],
    aggfunc="count",
    margins=True,              # Adds totals
    margins_name='Total',      # Name of the margins/total column
    dropna=False               # Include NaN values in the table
)

NameError: name 'newName1522' is not defined

In [179]:
pd.pivot_table(
    merged_df,
    values=UID,
    index=[newName2223],
    columns=['collection'],
    aggfunc="count",
    margins=True,              # Adds totals
    margins_name='Total',      # Name of the margins/total column
    dropna=False               # Include NaN values in the table
)

collection,QC_2023,coll_1522_1,coll_1622_1,coll_1622_2,coll_1622_3,coll_2023_1,coll_2023_2,Total
TNT_stratum_2023,,,,,,,,
1,254,315,768,1127,1831,218,356,4869
2,456,74,183,467,764,87,995,3026
3,259,23,49,221,311,53,549,1465
Total,969,412,1000,1815,2906,358,1900,9360


In [180]:
# Remove rows where the stratum has NaN values - outside the country
#merged_df = merged_df.dropna(subset=['TNT_stratum_2223'])

In [181]:
merged_df['sampling'].value_counts(dropna=False)

sampling
ceo_1522    6133
ceo_2023    3227
Name: count, dtype: int64

In [182]:
merged_df['collection'].value_counts(dropna=False)

collection
coll_1622_3    2906
coll_2023_2    1900
coll_1622_2    1815
coll_1622_1    1000
QC_2023         969
coll_1522_1     412
coll_2023_1     358
Name: count, dtype: int64

In [183]:
# Check if the lengths of merged_df and ceo are the same
if len(merged_df) == len(points):
    print("The lengths of merged_df and ceo are the same.")
else:
    print("The lengths of merged_df and ceo are different.")

# Optionally, you can print the lengths for clarity
print("Length of merged_df:", len(merged_df))
print("Length of ceo:", len(points))

The lengths of merged_df and ceo are different.
Length of merged_df: 9360
Length of ceo: 9359


#### export

In [184]:
out_file =  '/home/sepal-user/module_results/esbae/COG/COG_all_ceo_2015_2022_2023_prov_eSBAE_TNT.csv'
#sampled_df.to_csv(out_file,index=False)
merged_df.to_csv(out_file,index=False)